In [ ]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
from category_encoders import BinaryEncoder, cat_boost
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### SQL Queries

In [ ]:
joined_table = '''
CREATE TABLE joined_table AS
SELECT
    oh.store_id,
    oh.order_id,
    product_id,
    price,
    profit,
    delivery_distance,
    DATE_CREATE as date_create,
    order_start_prepare,
    planned_prep_time,
    order_ready,
    order_pickup,
    region_id,
    STATUS_ID as status_id
FROM
    order_history as oh
    LEFT JOIN order_props_value_pivoted as opvp on opvp.order_id = oh.order_id
    LEFT JOIN order_busket as ob on ob.order_id = oh.order_id
    and ob.store_id = oh.store_id;
'''
order_props_value_pivoted = '''
CREATE VIEW order_props_value_pivoted AS
SELECT
    opv.order_id,
    MAX(CASE WHEN op.ID = 11 THEN opv.VALUE END) AS region_id,
    MAX(CASE WHEN op.ID = 95 THEN opv.VALUE END) AS order_ready,
    MAX(CASE WHEN op.ID = 97 THEN opv.VALUE END) AS order_start_prepare,
    MAX(CASE WHEN op.ID = 65 THEN opv.VALUE END) AS delivery_distance,
    MAX(CASE WHEN op.ID = 18 THEN opv.VALUE END) AS order_pickup,
    MAX(CASE WHEN op.ID = 77 THEN opv.VALUE END) AS profit
FROM
    order_props_value opv
    LEFT JOIN order_props op ON opv.ORDER_PROPS_ID = op.ID
GROUP BY
    opv.order_id;
'''

# Reading from database and prepare target feature

In [ ]:
from sqlalchemy import create_engine, text

session_engine = create_engine('sqlite:///../F24.ML.Assignment.One.data.db')

with session_engine.connect() as conn: 
    original_orders = pd.read_sql_query(text('SELECT * FROM joined_table'), con=conn) 
    
original_orders.to_csv('../content/full_orders_df.csv', index=False)

In [ ]:
print("Before imputing. Num of nan's in price: ", original_orders['price'].isna().sum())
original_orders['imputed_price'] = original_orders.groupby(['store_id', 'product_id'])['price'].transform(lambda x: x.fillna(x.max()))
print("After imputing. Num of nan's in imputed_price: ", original_orders['imputed_price'].isna().sum())

aggregated_df = original_orders.groupby(['store_id', 'order_id'])[original_orders.columns].agg(
    products_count=('product_id', 'count'),
    order_price=('imputed_price', 'sum'),
    max_price=('imputed_price', 'max'),
    min_price=('imputed_price', 'min'),
    avg_price=('imputed_price', 'mean')
).reset_index()

print('Number of duplicates in orders (the same product in price, another feats., but other product_id): ', 
      original_orders.duplicated(subset=original_orders.columns.difference(['product_id'])).sum(),
      '\nShape of dataset: ', original_orders.shape)
original_orders = original_orders.drop_duplicates(subset=original_orders.columns.difference(['product_id']))
print('After drop duplicates: ', original_orders.shape)

unique_products_by_store = original_orders.groupby('store_id')['product_id'].nunique().reset_index()
unique_products_by_store.columns = ['store_id', 'unique_products_sold_by_store']

aggregated_df = pd.merge(aggregated_df, unique_products_by_store, on='store_id', how='left')

unique_orders = original_orders.drop_duplicates(subset=['store_id', 'order_id']).drop(columns=['product_id','price','imputed_price'])

df = pd.merge(unique_orders, aggregated_df, on=['store_id', 'order_id'], how='left')
df.drop(columns=['order_id'], inplace=True)

Before imputing. Num of nan's in price:  135275
After imputing. Num of nan's in imputed_price:  2065
Number of duplicates in orders (the same product in price, another feats., but other product_id):  72742 
Shape of dataset:  (1351692, 14)
After drop duplicates:  (1278950, 14)


In [ ]:
date_cols = ['date_create', 'order_start_prepare', 'order_ready'] 
for col_name in date_cols:
    df[col_name] = pd.to_datetime(df[col_name], format='mixed')
df['order_pickup'] = pd.to_datetime(df['order_pickup'], dayfirst=True)

df = df.sort_index(ascending=False)
df.to_csv('../content/aggregated_df.csv', index=False)